In [23]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords,gutenberg
from nltk.stem.porter import PorterStemmer
import spacy

In [24]:
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

In [25]:
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)

In [26]:
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text

In [27]:
persuasion_doc = text_cleaner(persuasion)
alice_doc = text_cleaner(alice)

In [28]:
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [29]:
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

In [30]:
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"([, Alice, 's, Adventures, in, Wonderland, by,...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


In [32]:
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()
df = pd.DataFrame(columns=common_words)
df['text_sentence'] = sentences[0]
df["text_sentence"] = df["text_sentence"].str.lower()
#df["text_sentence"] = df["text_sentence"].apply(ps.stem)
df.head(3)

,effort,declaration,thing,cruel,pause,respectability,afflict,tread,move,ears--,...,dab,ridge,difficulty,phoo,climate,residence,authorise,money,stick,text_sentence
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
from collections import Counter
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]

In [12]:
# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

In [13]:
# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

In [14]:
# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [15]:
# Create our data frame with features. This can take a while to run.
#word_counts = bow_features(sentences, common_words)

In [22]:
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()
df = pd.DataFrame(columns=common_words)
df['text_sentence'] = sentences[0]
df["text_sentence"] = df["text_sentence"].str.lower()
#df["text_sentence"] = df["text_sentence"].apply(ps.stem)
df.head(3)

,effort,declaration,thing,cruel,pause,respectability,afflict,tread,move,ears--,...,dab,ridge,difficulty,phoo,climate,residence,authorise,money,stick,text_sentence
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
